In [ ]:
import os
import numpy as np
import rasterio


In [ ]:
def read_rasters(file_paths):
    rasters = []
    for file_path in file_paths:
        with rasterio.open(file_path) as src:
            rasters.append(src.read(1))
    return rasters, src.meta

def weighted_sum_rasters(rasters, weights):
    weighted_sum = np.zeros_like(rasters[0], dtype=float)
    for raster, weight in zip(rasters, weights):
        weighted_sum += raster * weight
    return weighted_sum

def save_raster(output_path, data, meta):
    meta.update({"dtype": 'float32'})
    with rasterio.open(output_path, 'w', **meta) as dst:
        dst.write(data.astype('float32'), 1)

def read_config(config_file):
    with open(config_file, 'r') as f:
        lines = f.readlines()
    
    config = {}
    for line in lines:
        key, value = line.strip().split(' ', 1)
        config[key] = value
    
    return config

def main(config_file):
    # Read the configuration from the text file
    config = read_config(config_file)
    
    raster_folder = config['raster_folder']
    save_folder = config['save']
    
    # Read the raster file names and their weights
    raster_files = []
    weights = []
    
    for key, value in config.items():
        if key not in ['raster_folder', 'save']:
            raster_files.append(os.path.join(raster_folder, f"{key}.tif"))
            weights.append(float(value))
    
    # Check if the number of weights matches the number of raster files
    if len(raster_files) != len(weights):
        raise ValueError("The number of weights must match the number of raster files.")
    
    # Read the rasters and their metadata
    rasters, meta = read_rasters(raster_files)
    
    # Calculate the weighted sum of the rasters
    weighted_raster_sum = weighted_sum_rasters(rasters, weights)
    
    # Define the output file path
    output_file = os.path.join(save_folder, 'weighted_sum_raster.tif')
    
    # Save the result
    save_raster(output_file, weighted_raster_sum, meta)
    
    print("Weighted raster calculation completed and saved successfully.")


In [2]:
# Example usage(change the path to the configuration file)
config_file = r'C:\Users\Zhimin.Chen\Desktop\ZChen\tsunami\Tsunami_Risk_Zhimin\cal_weight.txt'
main(config_file)

Weighted raster calculation completed and saved successfully.
